In [1]:
!pip install -U pymgrid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 39.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.0 MB/s eta 0:00:00
  Created wheel for pymgrid: filename=pymgrid-1.2.2-py3-none-any.whl size=3492845 sha256=959796ec956f8e5f26d2e7c4f02b62083d55b24039f64e0efc0e542fedbb76ef
  Stored in directory: /root/.cache/pip/wheels/aa/9c/34/d880763dcc693c67448303756a6e1adb899bdc0d5080643030
Successfully built pymgrid


In [2]:
import numpy as np
from pymgrid import Microgrid
from pymgrid.modules import BatteryModule, LoadModule, RenewableModule, GridModule

battery = BatteryModule(min_capacity=0,
                        max_capacity=100,
                        max_charge=50,
                        max_discharge=50,
                        efficiency=1.0,
                        init_soc=0.5)

# Using random data
renewable = RenewableModule(time_series=50*np.random.rand(100))

load = LoadModule(time_series=60*np.random.rand(100))

grid = GridModule(max_import = 100, max_export=100, time_series=np.random.rand(100,3))

microgrid = Microgrid([battery, ("pv", renewable), load, grid])

In [3]:
microgrid

Microgrid([load x 1, pv x 1, balancing x 1, battery x 1, grid x 1])

In [4]:
microgrid.fixed

{
  "load": "[LoadModule(time_series=<class 'numpy.ndarray'>, forecaster=NoForecaster, forecast_horizon=0, forecaster_increase_uncertainty=False, raise_errors=False)]"
}

In [5]:
microgrid.flex

{
  "pv": "[RenewableModule(time_series=<class 'numpy.ndarray'>, raise_errors=False, forecaster=NoForecaster, forecast_horizon=0, forecaster_increase_uncertainty=False, provided_energy_name=renewable_used)]",
  "balancing": "[UnbalancedEnergyModule(raise_errors=False, loss_load_cost=10.0, overgeneration_cost=2.0)]"
}

In [6]:
microgrid.controllable

{
  "battery": "[BatteryModule(min_capacity=0, max_capacity=100, max_charge=50, max_discharge=50, efficiency=1.0, battery_cost_cycle=0.0, battery_transition_model=None, init_charge=None, init_soc=0.5, raise_errors=False)]",
  "grid": "[GridModule(max_import=100, max_export=100)]"
}

In [7]:
empty_action=microgrid.get_empty_action
print(empty_action)

<bound method Microgrid.get_empty_action of Microgrid([load x 1, pv x 1, balancing x 1, battery x 1, grid x 1])>


In [8]:
microgrid.reset()
microgrid.state_series.to_frame()

0
load    0 load_current         -39.997473
pv      0 renewable_current     14.306967
battery 0 soc                    0.500000
          current_charge        50.000000
grid    0 import_price_current   0.996086
          export_price_current   0.769397
          co2_per_kwh_current    0.573774
          grid_status_current    1.000000

In [9]:
load = -1.0 * microgrid.modules.load.item().current_load
pv = microgrid.modules.pv.item().current_renewable

In [10]:
net_load = load + pv

if net_load > 0:
    net_load = 0.0

In [11]:
battery_discharge = min(-1*net_load, microgrid.modules.battery.item().max_production)
net_load += battery_discharge

In [12]:
grid_import = min(-1*net_load, microgrid.modules.grid.item().max_production)

In [13]:
control = {"battery" : [battery_discharge],
           "grid": [grid_import]}

control

{'battery': [25.690506262325325], 'grid': [-0.0]}

In [15]:
for i in range(10):
  load = -1.0 * microgrid.modules.load.item().current_load
  pv = microgrid.modules.pv.item().current_renewable
  net_load = load + pv

  if net_load > 0:
    net_load = 0.0

  battery_discharge = min(-1*net_load, microgrid.modules.battery.item().max_production)
  net_load += battery_discharge

  grid_import = min(-1*net_load, microgrid.modules.grid.item().max_production)
  control = {"battery" : [battery_discharge],
           "grid": [grid_import]}
  microgrid.run(control, normalized=False)

In [16]:
microgrid.get_log()

module_name     load                             pv              \
module_number      0                              0               
field         reward   load_met load_current reward curtailment   
0                0.0  39.997473   -39.997473    0.0    0.000000   
1                0.0   6.354509    -6.354509    0.0    4.988064   
2                0.0   7.853697    -7.853697    0.0   19.712041   
3                0.0  19.318836   -19.318836    0.0   16.654612   
4                0.0  39.693860   -39.693860    0.0    0.000000   
5                0.0  50.790374   -50.790374    0.0    0.000000   
6                0.0  33.195441   -33.195441    0.0    1.046046   
7                0.0  51.267149   -51.267149    0.0    0.000000   
8                0.0  23.090269   -23.090269    0.0    0.000000   
9                0.0  19.007274   -19.007274    0.0    0.000000   

module_name                                    balancing            \
module_number                                          0             
field         renewable_used renewable_current    reward loss_load   
0                  14.306967         14.306967      -0.0       0.0   
1                   6.354509         11.342573      -0.0       0.0   
2                   7.853697         27.565738      -0.0       0.0   
3                  19.318836         35.973448      -0.0       0.0   
4                  21.155323         21.155323      -0.0       0.0   
5                  49.038210         49.038210      -0.0       0.0   
6                  33.195441         34.241487      -0.0       0.0   
7                  24.046595         24.046595      -0.0       0.0   
8                  19.605876         19.605876      -0.0       0.0   
9                  17.158901         17.158901      -0.0       0.0   

module_name                   ...                 grid                      \
module_number                 ...                    0                       
field         overgeneration  ... export_price_current co2_per_kwh_current   
0                        0.0  ...             0.769397            0.573774   
1                        0.0  ...             0.699834            0.661168   
2                        0.0  ...             0.792299            0.518717   
3                        0.0  ...             0.788187            0.411569   
4                        0.0  ...             0.181629            0.321319   
5                        0.0  ...             0.186904            0.417291   
6                        0.0  ...             0.236600            0.916832   
7                        0.0  ...             0.091296            0.463653   
8                        0.0  ...             0.313669            0.047340   
9                        0.0  ...             0.095530            0.238250   

module_name                          balance                                \
module_number                              0                                 
field         grid_status_current     reward overall_provided_to_microgrid   
0                             1.0   0.000000                     39.997473   
1                             1.0   0.000000                      6.354509   
2                             1.0   0.000000                      7.853697   
3                             1.0   0.000000                     19.318836   
4                             1.0   0.000000                     39.693860   
5                             1.0   0.000000                     50.790374   
6                             1.0   0.000000                     33.195441   
7                             1.0 -21.308439                     51.267149   
8                             1.0  -1.749919                     23.090269   
9                             1.0  -0.446725                     19.007274   

module_name                                    \
module_number                                   
field         overall_absorbed_from_microgrid   
0                                   39.9